In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19,preprocess_input
from Segmentation import *
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from warnings import filterwarnings
from skimage.transform import rescale
filterwarnings('ignore')

2022-11-15 09:17:43.669131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 09:17:43.738856: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-15 09:17:43.757778: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 09:17:44.098388: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [15]:
def preprocessing_function(img):
    
    # Convert to BGR image
    img_bgr = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    
    # Removing Shadows from the Images    
#     img_rgb_plane = cv2.split(img_bgr) # Spliting the color channels
#     result_norm_planes = []
#     for plane in img_rgb_plane:
#         plane = plane.astype("uint8")
#         dilated_img = cv2.dilate(plane, np.ones((25, 25), np.uint8)) # dilate image
#         bg_img = cv2.medianBlur(dilated_img, 85)  # blur background
#         diff_img = 255 - cv2.absdiff(plane, bg_img)  # find the difference in the image pixels
#         norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
#         result_norm_planes.append(norm_img)
#     result_norm = cv2.merge(result_norm_planes)  # Merging all normalized color channels

    blank_mask = np.zeros(img_bgr.shape, dtype=np.uint8)
    original = img_bgr.copy()
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower = np.array([18, 42, 69])
    higher = np.array([179,255,255])
    cnts = cv2.findContours(img_bgr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    for c in cnts:
        cv2.drawContours(blank_mask,[c], -1, (255,255,255), -1)
        break

    result = cv2.bitwise_and(original,blank_mask)

    cv2.imshow('result', result)
                    
    
    # Convert the Normalized image to GrayScale
    image_grayscale = cv2.cvtColor(result_norm,cv2.COLOR_BGR2GRAY).astype("uint8")
    
    # Equalize Image for exposure Equalization
    image_equalize = cv2.equalizeHist(image_grayscale)

    # Blur Image using Gaussian Filter
    image_blur = cv2.GaussianBlur(image_equalize,(15,15),0)
    image_blur = image_blur.astype("uint8")
    
    # Impliment Otsu's Threshold - Detect Boundaries
    res,image_threshold = cv2.threshold(image_blur,0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # Perform Morphological Transform to eliminate holes within the image regions
    kernel = np.ones((50,50), np.uint8)
    image_morph = cv2.morphologyEx(image_threshold,cv2.MORPH_OPEN, kernel)
    image_morph = cv2.cvtColor(image_morph,cv2.COLOR_GRAY2RGB) 
    
    # Smoothing edges:
    kernel = np.ones((15,15), np.uint8)
    image_smooth = cv2.morphologyEx(image_morph, cv2.MORPH_OPEN, kernel)
    
    # Merge Orginal Image with masked area
    image = cv2.cvtColor(img_bgr,cv2.COLOR_BGR2RGB).astype("uint8")
    image_bitwise = cv2.bitwise_or(image_smooth,image)
    
#     # Preprocess Image if Pretrained Model used.
    image_bitwise = preprocess_input(image_bitwise)
    
    return image_bitwise.astype('float')

In [16]:
training_generator = ImageDataGenerator(
                                   featurewise_center=True,
                                   featurewise_std_normalization=True,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=10,
                                   shear_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='nearest',
                                   preprocessing_function=preprocessing_function,
#                                    rescale =1./255
                                    )

validation_generator = ImageDataGenerator(preprocessing_function=preprocessing_function,
                                         rescale = 1./255)

In [17]:
batch_size = 64
target_size = (224,224)
class_mode = 'categorical'

In [18]:
batch_size = 64

path_train = 'New_Dataset_plant/Orginal/train/'
path_valid = 'New_Dataset_plant/Orginal/valid/'

training_data = training_generator.flow_from_directory(path_train,
                                                      batch_size=batch_size,
                                                      target_size=target_size,
                                                      class_mode=class_mode,
                                                      shuffle=True)

validating_data = validation_generator.flow_from_directory(path_valid,
                                                          batch_size = batch_size,
                                                          target_size = target_size,
                                                          class_mode = class_mode,
                                                          shuffle=True)

Found 67970 images belonging to 32 classes.
Found 15435 images belonging to 32 classes.


In [19]:
train_img, labels = training_data.next()
dicte = training_data.class_indices
classes = list(dicte.keys())
print(classes)

error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/contours.cpp:195: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'


In [ ]:
def plot_img(img, label):
    for i,imgs in enumerate(img):
        image = imgs
        pos = np.argmax(label[i])
        plt.figure(figsize=(5,5))
        plt.imshow(image)
        plt.title(classes[pos])
        plt.show()
    
plot_img(train_img,labels)